In [ ]:
!pip install -r requirements.txt

In [1]:
import textToVector
import searchVector
import rag

## **Crear una DB vectorial**

In [ ]:
# Obtendras el nombre de una DB vectorial a partir de un archivo de texto, una lista de textos o un archivo PDF.
# Podrás usar este nombre para buscar en la DB vectorial más tarde.

nameDB = textToVector.textToDBVector(fileText = "constitución politica del Perú.pdf", name = "myDB")
print(nameDB)

## **Buscar en la DB vectorial**

In [3]:
# Obtendras los k textos más relacionados a text, con su score de similitud y cantidad de tokens.

text = "horas de trabajo"

result = searchVector.searchText(text, nameDB, k =5)
for i in result:
    print(i[0])
    print(i[1])
    print("-"*100)


1.0316013415247607
{'text': 'CONSTITUCIÓN POLITICA DEL PERÚ\n\nEl trabaj ador tiene derecho a una remuneración equitativa y suficiente, que procure, para él y su familia, el bienestar material y espiritual. El pago de la remuneraci ón y de los ben eficios sociales del trabajador tiene prioridad sobre cualquiera otra obligación del empleado r. Las rem uneraciones mínimas se  regulan por el Estado con participación de las organizaciones representativas de los trabajadores y de los empleadores. Artículo 25.- Jornada Ordinaria de trabajo La jornada ordinaria de t rabajo es de och o horas diarias o cuarenta y ocho h oras sem anales, como máximo. En caso de jornadas acumulativas o atípicas, el promedio de horas trabajadas en el período correspondiente no puede superar dicho máximo. Los trabajadores ti enen derecho a d escanso semanal y anual remunerados. Su disfrute y su comp ensación se regulan por ley o por convenio. Artículo 26.- Princi pios que regulan la relación laboral En la relación 

## **RAG**

In [4]:
# Obtendras una respuesta a partir de una pregunta, usando el nombre de la DB vectorial
# también obtendras los k textos más relacionados a la pregunta, y su cantidad de tokens.

respuesta = rag.responseAssistant([{"role": "user", "content": "cuantas horas como maximo a la semana se puede trabajar en perú?"}], nameDB)
for i in respuesta:
    print(f"{i}: \n{respuesta[i]}")
    print("-"*128)

role: 
assistant
--------------------------------------------------------------------------------------------------------------------------------
content: 
En Perú, la jornada ordinaria de trabajo es de un máximo de cuarenta y ocho horas semanales.
--------------------------------------------------------------------------------------------------------------------------------
context: 
1). CONSTITUCIÓN POLITICA DEL PERÚ

Primera.- Regímene s pensionari os de l os Servid ores Públicos Los nuevos regímenes social es obligatorios, que  sobre  materia de pensiones de los tr abajadores públic os, se establezcan,  no afectan los derechos legalment e obtenidos, en particular el correspondiente a  los regímenes de los d ecretos le yes 199 90 y 20530 y sus modificatoria s. Segunda. - Pago y reajuste  de  pensiones  que ad ministra el Estado El Estado garantiza el pago o portuno y el reajuste periódico de las pensiones que administr a, con arreglo a las previsiones presupuestarias que éste destin

## **RAG con salida de audio**

In [ ]:
!pip install playsound
!pip install pygame

In [5]:
# Funciones para guardar y reproducir audio. No es obligatorio usar estas funciones, ni estas librerías para que el LRAG funcione, pero pueden ser útiles.

import base64
import wave
import pygame
import os
import time

def base64_to_audio(base64_string, output_filename='output.wav'):
    """
    Convierte una cadena base64 a un archivo de audio WAV
    
    Args:
        base64_string (str): String en formato base64 que contiene los datos del audio
        output_filename (str): Nombre del archivo de salida (por defecto: 'output.wav')
    """
    try:
        # Decodificar el string base64
        audio_data = base64.b64decode(base64_string)
        
        # Crear y escribir el archivo WAV
        with wave.open(output_filename, 'wb') as wave_file:
            wave_file.setnchannels(1)  # Mono
            wave_file.setsampwidth(2)  # 2 bytes por muestra
            wave_file.setframerate(int(44100/2))  # Frecuencia de muestreo estándar
            wave_file.writeframes(audio_data)
        
    except Exception as e:
        print(f"Error al convertir el audio: {str(e)}")

def playAudio(filename):
    """
    Reproduce un archivo de audio WAV
    
    Args:
        filename (str): Ruta al archivo de audio a reproducir
    """
    try:
        # Inicializar pygame mixer
        pygame.mixer.init()
        
        # Cargar y reproducir el archivo
        pygame.mixer.music.load(filename)
        pygame.mixer.music.play()
        
        # Esperar a que termine la reproducción
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
            
        # Limpiar recursos
        pygame.mixer.quit()
        
    except Exception as e:

        print(f"Error al reproducir el audio: {str(e)}")

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
# Si deseas usar el audio, debes activar el audio_enabled = True
# Este código te permitirá interactuar con el LRAG obteniendo respuestas con audio.
historial = []
audio_enabled = True

while True:
    pregunta = input("Ingrese su pregunta: ")
    if pregunta.lower() in ["salir", "exit", "e", "terminar", "quit"]:
        break
    historial.append({"role": "user", "content": pregunta})
    print("Usuario: ", pregunta)
    respuesta = rag.responseAssistant(historial, nameDB, k = 3, audio_enabled = audio_enabled)
    print("Asistente: ", respuesta["content"])
    print("-"*128)
    historial.append({"role": "assistant", "audio" : {"id": respuesta["idAudio"]}})
    if respuesta["audio"] != None:
        base64_to_audio(respuesta["audio"])
        playAudio("output.wav")
        os.remove("output.wav")

Usuario:  Cual es tu función?
Asistente:  Mi función es responder preguntas y brindar información basada en el contexto proporcionado. En este caso, puedo responder preguntas relacionadas con la Constitución Política del Perú, referida al ámbito electoral y otros temas mencionados en los fragmentos.
--------------------------------------------------------------------------------------------------------------------------------
Usuario:  Qué me puedes decir sobre las lenguas originarias en Perú?
Asistente:  En el Perú, las lenguas originarias como el quechua, el aimara y demás lenguas aborígenes son reconocidas como idiomas oficiales en las zonas donde predominen, según la ley. Esto significa que, además del castellano, estas lenguas también son utilizadas en asuntos oficiales y tienen una importancia cultural y social significativa en el país.
--------------------------------------------------------------------------------------------------------------------------------
